# Introduction 
 The recent paideia project

In [ ]:
# Run the command 
# pip install --force-reinstall -r '..\paideia\requirements.txt'

In [2]:
pip install dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
    ERROR: Command errored out with exit status 1:
     command: /Users/jensbay/opt/anaconda3/envs/bigsleep/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/m7/fz3wxs8s1w348l1yjktbwwp80000gn/T/pip-install-eb0vbdib/dotenv_ff6efacff61443bab3385be62c636e8c/setup.py'"'"'; __file__='"'"'/private/var/folders/m7/fz3wxs8s1w348l1yjktbwwp80000gn/T/pip-install-eb0vbdib/dotenv_ff6efacff61443bab3385be62c636e8c/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/m7/fz3wxs8s1w348l1yjktbwwp80000gn/T/pip-pip-egg-info-fk6ntxkc
         cwd: /private/var/folders/m7/fz3wxs8s1w348l1yjktbwwp80000gn/T/pip-install-eb0vbdib/dotenv_ff6efacff61443bab3385be62c636e8c

In [1]:
import  pandas as pd
import  os
from    datetime import datetime
from    tqdm import tqdm
import  regex as re
import  time
import  numpy as np
import  random
import  pyperclip as py
# import  PyPDF2
import  openai 
import  regex as re
import  random 
import  os

import  utilities as my

from    dotenv import load_dotenv
load_dotenv()
openai.api_type     = os.getenv('OPENAI_TYPE')
openai.api_base     = os.getenv('OPENAI_BASE')
openai.api_version  = os.getenv('OPENAI_VERSION')
openai.api_key      = os.getenv('OPENAI_KEY')

import tiktoken

def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      engine = 'text-embedding-ada-002',
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]

ModuleNotFoundError: No module named 'dotenv'

In [9]:
def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      engine = 'text-embedding-ada-002',
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]
  
def summary(author,text,title, written_date):
    text = 'In their article {author}, writes with the title: "{title}" which was published on {written_date}. The following: \n {text}'.format(author=author,text=text,title=title,written_date=written_date)
    return text 


In [10]:
def new_func():
    # now lets first make a script which takes the clipboard and converts it into a list of strings
    # the first line is the title
    # the second line is the author
    # the third line is the date
    # the fourth line is the url
    # from there the rest is text
    # the text is split by the new line character
    # then if the line contains less than 20 characters it is removed
    # then we create a dataframe with the title, author, date, and text
    # we then save that to a csv file which is called master, which will store all the articles 
    # lets start by loading in the clipboard
    # we can use pyperclip for this
    x = py.paste()

    # now lets split the text by the new line character
    x = x.split('\r\n')

    runable = True

    title = x[0]
    author = x[1]
    date = x[2]
    url = x[3]

    if len(date.split('-')) < 2:
        runable = False
    
    # advertise sentences
    advertisements = ['Subscribe here to ',
                  ]

    if runable:
    # now lets remove all the lines which have less than 20 characters
        text = x[4:]

    # now lets create a dataframe with the title, author, date, and text
        df = pd.DataFrame(columns=['title', 'author', 'date', 'url', 'text'])
        df['text'] = text

        df['title'] = title
        df['author'] = author
        df['date'] = date
        df['url'] = url

    # now lets get the length of the text column in a sepearet column
        df['length'] = df['text'].apply(lambda x: len(x))

        def num_tokens_from_string(string, encoding_name):
            """Returns the number of tokens in a text string."""
            encoding = tiktoken.get_encoding(encoding_name)
            num_tokens = len(encoding.encode(string))
            return num_tokens

        df['token_count'] = df['text'].apply(lambda text: num_tokens_from_string(text,'cl100k_base'))

    # now lets go through the dataframe and join on the text cells until they reach a sum of the token count of 200
    # then we will add that to a list and then create a new dataframe with the text and the token count

        new_strings = []

        current_tokens = 0

        y = ''

        for i in range(len(df)):
            x = df.at[i, 'token_count']
        
            if current_tokens + x < 300:
                y += df.at[i, 'text']
                current_tokens += x
            else:
                new_strings.append(y)
                y = ''
                current_tokens = 0
        
    # now lets overwrite the df with the new_strings values but the same title, author, date, and url
        df = pd.DataFrame(columns=['title', 'author', 'date', 'url', 'text'])
        df['text'] = new_strings
        df['author'] = author
        df['date'] = date
        df['url'] = url
        df['title'] = title

        new_line = [title, author, date, url, '[Insert Summary Here]']
        df.loc[len(df)] = new_line

        df['one_line'] = ''
        df['question'] = ''
    
        df['summary'] = df.apply(lambda x: summary(x['author'],x['text'],x['title'],x['date']),axis=1)
    
        df['embedding'] = df['summary'].apply(lambda x: get_embedding(x))
    else:
        print('The date is not in the correct format')
        df = ''
    
    
    return df

df = new_func()

In [11]:
df

,title,author,date,url,text,one_line,question,summary,embedding
0,OpenAI inks deal with Axel Springer on licensi...,Kyle Wiggers,13-12-2023,https://techcrunch.com/2023/12/13/openai-inks-...,"Many, if not most, generative AI tech vendors ...",,,"In their article Kyle Wiggers, writes with the...","[-0.002548110904172063, -0.036037568002939224,..."
1,OpenAI inks deal with Axel Springer on licensi...,Kyle Wiggers,13-12-2023,https://techcrunch.com/2023/12/13/openai-inks-...,Outside of the publishers tapping generative A...,,,"In their article Kyle Wiggers, writes with the...","[0.001979380613192916, -0.024551965296268463, ..."
2,OpenAI inks deal with Axel Springer on licensi...,Kyle Wiggers,13-12-2023,https://techcrunch.com/2023/12/13/openai-inks-...,[Insert Summary Here],,,"In their article Kyle Wiggers, writes with the...","[0.002085978863760829, -0.025954222306609154, ..."


In [13]:
master_path = r'csv_files\paideia_master.xlsx'

master = pd.read_excel(master_path)

master.tail(3)

,title,author,date,url,text,one_line,question,summary,token_count,embedding,prompt_similarity
67,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,"So set a “proximate objective.” Yes, that soun...",NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.018216855823993683, -0.01605622097849846, ...",NaN
68,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Guiding Policy: How can you apply your str...,NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.017031535506248474, -0.006848321296274662,...",NaN
69,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,[Insert Summary Here],NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.014179251156747341, 0.005765299312770367, ...",NaN


In [15]:
master = pd.read_excel(master_path)

# now lets concatenate the df to the master
master = pd.concat([master, df], ignore_index=True)

master.to_excel(master_path, index=False)


In [16]:
master

,title,author,date,url,text,one_line,question,summary,token_count,embedding,prompt_similarity
0,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"You’d be quick to conclude then, that, those w...",Sangeet Paul Choudary argues that the real thr...,NaN,"In their article Sangeet Paul Choudary, writes...",214.0,"[-0.011768677271902561, -0.02153008058667183, ...",0.763318
1,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,This issue right here is Part 1 of a three-par...,"Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",226.0,"[-0.009319336153566837, -0.016895689070224762,...",0.755640
2,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"Up until the start of the twentieth century, a...",Sangeet Paul Choudary argues that augmentation...,NaN,"In their article Sangeet Paul Choudary, writes...",222.0,"[-0.008647583425045013, -0.013699556700885296,...",0.734889
3,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,When technology augments skilled work and enab...,"Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",209.0,"[-0.014073953963816166, -0.016499046236276627,...",0.752511
4,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"This is an early study, possibly with many cav...","Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",236.0,"[-0.013471231795847416, -0.02042117714881897, ...",0.753413
...,...,...,...,...,...,...,...,...,...,...,...
68,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Guiding Policy: How can you apply your str...,NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.017031535506248474, -0.006848321296274662,...",NaN
69,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,[Insert Summary Here],NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.014179251156747341, 0.005765299312770367, ...",NaN
70,OpenAI inks deal with Axel Springer on licensi...,Kyle Wiggers,13-12-2023,https://techcrunch.com/2023/12/13/openai-inks-...,"Many, if not most, generative AI tech vendors ...",,,"In their article Kyle Wiggers, writes with the...",NaN,"[-0.002548110904172063, -0.036037568002939224,...",NaN
71,OpenAI inks deal with Axel Springer on licensi...,Kyle Wiggers,13-12-2023,https://techcrunch.com/2023/12/13/openai-inks-...,Outside of the publishers tapping generative A...,,,"In their article Kyle Wiggers, writes with the...",NaN,"[0.001979380613192916, -0.024551965296268463, ...",NaN
